<a href="https://colab.research.google.com/github/ShivamTiwariCodes/association-rules-using-apriori-algorithm/blob/main/association-rules-using-apriori-algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

store_data = pd.read_csv('store_data.csv', header = None)
store_data.head

<bound method NDFrame.head of                  0                  1   ...       18         19
0            shrimp            almonds  ...  spinach  olive oil
1           burgers          meatballs  ...      NaN        NaN
2           chutney                NaN  ...      NaN        NaN
3            turkey            avocado  ...      NaN        NaN
4     mineral water               milk  ...      NaN        NaN
...             ...                ...  ...      ...        ...
7496         butter         light mayo  ...      NaN        NaN
7497        burgers  frozen vegetables  ...      NaN        NaN
7498        chicken                NaN  ...      NaN        NaN
7499       escalope          green tea  ...      NaN        NaN
7500           eggs    frozen smoothie  ...      NaN        NaN

[7501 rows x 20 columns]>

In [2]:
store_data = np.array(store_data)

In [3]:
def one_frequent(store_data):
    c1 = [] 
    rows,columns = store_data.shape 
    for i in range(0,rows):
      for j in range(0,columns):
            data_point =store_data[i,j]  
            if(type(data_point) == float): 
               break
            data_point = data_point.strip()
            data_point = data_point.lower()
            data_point  = frozenset([data_point])
            if data_point not in c1:
                c1.append(data_point)
    return c1

In [4]:
def get_candidates(item, k):
    res = [] 
    
    
    if k == 0:
        cmb =combinations(item, 2) 
        for set1, set2 in cmb:
            item = set1 | set2 
            res.append(item)
    else:    
        cmb=combinations(item, 2)
        for set1, set2 in cmb:       
            intersection = set1 & set2 
            if len(intersection) == k: 
                item = set1 | set2
                if item not in res: 
                    res.append(item)
    return res

In [5]:
def frequently_used_item(data, candi, min_support):
    
    n = data.shape[0] 
    items = [] 
    item_support = {} 
    item_count = {}

    for t in data:
        for item in candi:
            if item.issubset(t):
                if item not in item_count: 
                    item_count[item] = 1
                else: 
                    item_count[item] += 1    
    

    for item in item_count:
        support = (item_count[item] / n)
        if support >= min_support:
            items.append(item)
        
        item_support[item] = support
        
    return items, item_support

In [6]:
def get_Ap(data,c1, min_support):
    
    regular_item, item_support_dict = frequently_used_item(data, c1, min_support = 0.005)
    items1 = [regular_item]
    
    count = 0
    
    while len(items1[count]) > 0:

        regular_item = items1[count]
        candidate = get_candidates(regular_item, count)       
        regular_item, item_support = frequently_used_item(data, candidate, min_support = 0.005)

        items1.append(regular_item)
        item_support_dict.update(item_support)
        count += 1
        print(item_support_dict)

    return items1, item_support_dict

In [7]:
input = one_frequent(store_data)
items, item_support_dict = get_Ap(store_data, input, min_support = 0.005)

{frozenset({'shrimp'}): 0.07145713904812692, frozenset({'almonds'}): 0.020397280362618318, frozenset({'avocado'}): 0.03332888948140248, frozenset({'vegetables mix'}): 0.025729902679642713, frozenset({'green grapes'}): 0.009065457938941474, frozenset({'whole weat flour'}): 0.009332089054792695, frozenset({'yams'}): 0.011465137981602452, frozenset({'cottage cheese'}): 0.03186241834422077, frozenset({'energy drink'}): 0.026663111585121985, frozenset({'tomato juice'}): 0.030395947207039063, frozenset({'low fat yogurt'}): 0.07652313024930009, frozenset({'green tea'}): 0.13211571790427942, frozenset({'honey'}): 0.047460338621517134, frozenset({'salad'}): 0.004932675643247567, frozenset({'mineral water'}): 0.23836821757099053, frozenset({'salmon'}): 0.04252766297826956, frozenset({'antioxydant juice'}): 0.008932142381015865, frozenset({'frozen smoothie'}): 0.06332489001466471, frozenset({'spinach'}): 0.007065724570057326, frozenset({'olive oil'}): 0.0658578856152513, frozenset({'burgers'}): 0

In [8]:
def create_association_rules(frequent_items, item_support_dict, min_confidence):
    
    association_rules = []

    for idx, frequent_item in enumerate(frequent_items[1:(len(frequent_items) - 1)]):
        for frequent_set in frequent_item:
            
            subsets = [frozenset([item]) for item in frequent_set]
            rules, RHS = compute_confidence(frequent_items, item_support_dict, 
                                                  frequent_set, subsets, min_confidence)
            association_rules.extend(rules)
            
            if idx != 0:
                k = 0
                while len(RHS[0]) < len(frequent_set) - 1:
                    ck = get_candidates(RHS, k = k)
                    rules, RHS = compute_confidence(frequent_items, item_support_dict,
                                                          frequent_set, ck, min_confidence)
                    association_rules.extend(rules)
                    k += 1    
    
    return association_rules

In [9]:
def compute_confidence(frequent_items, item_support_dict, frequent_set, subsets, min_confidence):
  
    rules = []
    RHS = []
    
    for rhs in subsets:
        lhs = frequent_set - rhs
        confidence = item_support_dict[frequent_set] / item_support_dict[lhs]
        if confidence >= min_confidence:
            lift = confidence / item_support_dict[rhs]
            rules_info = lhs, rhs, confidence, lift
            rules.append(rules_info)
            RHS.append(rhs)
            
    return rules, RHS

In [10]:
association_rules = create_association_rules(items, item_support_dict, min_confidence = 0.02)

for association_rule in association_rules:
  print(association_rule[0], "->", association_rule[1], association_rule[2], association_rule[3])

frozenset({'shrimp'}) -> frozenset({'low fat yogurt'}) 0.0876865671641791 1.145883171251755
frozenset({'low fat yogurt'}) -> frozenset({'shrimp'}) 0.0818815331010453 1.1458831712517552
frozenset({'shrimp'}) -> frozenset({'green tea'}) 0.16044776119402984 1.2144486949711584
frozenset({'green tea'}) -> frozenset({'shrimp'}) 0.08678102926337034 1.2144486949711584
frozenset({'shrimp'}) -> frozenset({'mineral water'}) 0.3302238805970149 1.385351973354703
frozenset({'mineral water'}) -> frozenset({'shrimp'}) 0.09899328859060404 1.385351973354703
frozenset({'shrimp'}) -> frozenset({'frozen smoothie'}) 0.10261194029850745 1.6204045561665357
frozenset({'frozen smoothie'}) -> frozenset({'shrimp'}) 0.11578947368421054 1.620404556166536
frozenset({'shrimp'}) -> frozenset({'olive oil'}) 0.11380597014925371 1.7280538098978788
frozenset({'olive oil'}) -> frozenset({'shrimp'}) 0.12348178137651822 1.7280538098978788
frozenset({'green tea'}) -> frozenset({'almonds'}) 0.03834510595358224 1.87991267815568